In [1]:
import os
from dotenv import load_dotenv
from huggingface_hub import login
import pandas as pd

/Users/andrestrepo/Documents/repos_personal/poc-summary/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
os.chdir('..')

# cargar y crear datasets de test y entrenamiento en formato json

In [2]:
load_dotenv()
login(token = os.environ.get("HF_TOKEN") )
dataset_name = "AndresR2909/youtube_transcriptions_summaries_2025_gpt4.1"

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df_test = pd.read_parquet(f"hf://datasets/{dataset_name}/" + splits["test"])
df_train = pd.read_parquet(f"hf://datasets/{dataset_name}/" + splits["train"])

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2004 entries, 0 to 2003
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   channel_name       2004 non-null   object 
 1   video_id           2004 non-null   object 
 2   source             2004 non-null   object 
 3   publish_date       1452 non-null   object 
 4   duration           2004 non-null   float64
 5   last_update_date   2004 non-null   object 
 6   title              2004 non-null   object 
 7   text               2004 non-null   object 
 8   year               2004 non-null   int64  
 9   month              1452 non-null   float64
 10  number_of_tokenks  2004 non-null   int64  
 11  prompt             2004 non-null   object 
 12  summary            2004 non-null   object 
 13  key_terms          2004 non-null   object 
 14  __index_level_0__  2004 non-null   int64  
dtypes: float64(2), int64(3), object(10)
memory usage: 235.0+ KB


In [6]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   channel_name       221 non-null    object 
 1   video_id           221 non-null    object 
 2   source             221 non-null    object 
 3   publish_date       158 non-null    object 
 4   duration           221 non-null    float64
 5   last_update_date   221 non-null    object 
 6   title              221 non-null    object 
 7   text               221 non-null    object 
 8   year               221 non-null    int64  
 9   month              158 non-null    float64
 10  number_of_tokenks  221 non-null    int64  
 11  prompt             221 non-null    object 
 12  summary            221 non-null    object 
 13  key_terms          221 non-null    object 
 14  __index_level_0__  221 non-null    int64  
dtypes: float64(2), int64(3), object(10)
memory usage: 26.0+ KB


In [9]:
df_test['channel_name'].unique()

array(['ARENA ALFA ', 'Bitcoin hoy', 'Bolsas hoy | Esteban Pérez',
       'Bolsas hoy | Invierte y Crece ', 'Esteban Perez ',
       'USACRYPTONOTICIAS'], dtype=object)

In [11]:
df_test[df_test['channel_name']== 'Bolsas hoy | Invierte y Crece '].iloc[-1]['prompt']

'Actúa como un experto en trading y análisis de mercados financieros.\n\nINSTRUCCIONES:\n1. Analiza el texto proporcionado entre las líneas de guiones.\n2. Elabora un informe estructurado siguiendo exactamente el formato solicitado.\n3. Utiliza un lenguaje claro, conciso y relevante para inversores.\n4. No inventes información; limita tu análisis únicamente al contenido del texto.\n\nFORMATO DEL INFORME:\n- **Introducción:** Presenta una visión general del tema tratado.\n- **Puntos clave:** Resume los aspectos más importantes en formato de viñetas.\n- **Conclusión:** Ofrece un cierre que sintetice el análisis realizado.\n- **Activos recomendados:** Extrae y lista, en una sección aparte, todos los activos mencionados como opciones de inversión.\n\nTexto a analizar:\n------------\nexpectativas en las bolsas hoy con el ojo puesto el bitcoin. muy buenos días arrancan las bolsas hoy martes 9 de enero son las 9:11 minutos de madrid este primer análisis que vamos a mantener a partir de ahora 


---

# Workshop: Fine-Tuning Llama 3.2-3B-Instruct for Summarization with YouTube Transcriptions

## Objetivo

Este workshop te guiará en el proceso de **Supervised Fine-Tuning (SFT)** de un modelo Llama 3.2-3B-Instruct, adaptándolo para tareas de resumen automático a partir de transcripciones de videos de YouTube. Aprenderás a preparar datos reales, ajustar el modelo y evaluar su desempeño en generación de resúmenes.

---

## 1. Descripción del Dataset

El dataset **AndresR2909/youtube_transcriptions_summaries_2025_gpt4.1** contiene transcripciones y resúmenes generados para videos de YouTube, ideal para entrenar modelos de resumen automático.

### Características principales:
- **Volumen:** 221 ejemplos, cada uno con transcripción, prompt e instructivo, y resumen.
- **Columnas relevantes:**
  - `text`: Transcripción completa del video (input).
  - `prompt`: Instrucción o pregunta para el modelo (instruction).
  - `summary`: Resumen generado (output).
- **Otras columnas:** Información de canal, fecha, duración, título, términos clave, etc.

### Ejemplo de estructura para fine-tuning:


In [ ]:
{
  "instruction": "<prompt>",
  "input": "<text>",
  "output": "<summary>"
}



---

## 2. Descripción de la Tarea

### Objetivo:
Ajustar el modelo **Llama 3.2-3B-Instruct** para que genere resúmenes precisos y contextuales a partir de transcripciones de videos, usando instrucciones específicas.

### Pasos del Workshop:

1. **Exploración y Preprocesamiento de Datos:**
   - Analizar la estructura del DataFrame.
   - Convertir los datos a formato JSON compatible con litgpt (`instruction`, `input`, `output`).

2. **Selección del Modelo:**
   - Usar [`meta-llama/Llama-3.2-3B-Instruct`](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct), un modelo eficiente y potente para tareas de generación de texto.

3. **Configuración del Fine-Tuning:**
   - Definir hiperparámetros: learning rate, batch size, epochs, etc.
   - Entrenar el modelo con litgpt usando el dataset procesado.

4. **Evaluación e Iteración:**
   - Validar el modelo con un subconjunto de datos.
   - Analizar la calidad de los resúmenes y ajustar el proceso si es necesario.

5. **Consideraciones de Despliegue:**
   - Integrar el modelo en un sistema de generación de resúmenes.
   - Optimizar inferencia y discutir aspectos éticos.

### Resultado Esperado:
- Experiencia práctica en preprocesamiento, fine-tuning y evaluación de modelos LLM.
- Un modelo ajustado capaz de generar resúmenes efectivos a partir de transcripciones de YouTube.
- Comprensión de cómo adaptar LLMs a tareas específicas usando SFT.

---

## Recursos

- **Modelo:** [meta-llama/Llama-3.2-3B-Instruct](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct)
- **Dataset:** [AndresR2909/youtube_transcriptions_summaries_2025_gpt4.1](https://huggingface.co/datasets/AndresR2909/youtube_transcriptions_summaries_2025_gpt4.1)

---

¿Necesitas el código para convertir tu DataFrame a este formato JSON?

In [13]:
import json

# Leer la instrucción desde el archivo
with open('../app/prompts/v3_summary_expert.txt', 'r', encoding='utf-8') as f:
    instruction_template = f.read()

print(instruction_template)

Actúa como un experto en trading y análisis de mercados financieros.

INSTRUCCIONES:
1. Analiza el texto proporcionado entre las líneas de guiones.
2. Elabora un informe estructurado siguiendo exactamente el formato solicitado.
3. Utiliza un lenguaje claro, conciso y relevante para inversores.
4. No inventes información; limita tu análisis únicamente al contenido del texto.

FORMATO DEL INFORME:
- **Introducción:** Presenta una visión general del tema tratado.
- **Puntos clave:** Resume los aspectos más importantes en formato de viñetas.
- **Conclusión:** Ofrece un cierre que sintetice el análisis realizado.
- **Activos recomendados:** Extrae y lista, en una sección aparte, todos los activos mencionados como opciones de inversión.

Texto a analizar:
------------
{context}
------------

Recuerda: Sigue el formato solicitado y asegúrate de que la información sea precisa y útil para inversores.



In [17]:
ls

app/              data/             notebooks/        requirements.txt


In [18]:
def convert_to_json_format(df,instruction):
    return [
        {
            "instruction": instruction,
            "input": row["text"],
            "output": row["summary"]
        }
        for _, row in df.iterrows()
    ]

# Convertir train y test
train_data_litgpt = convert_to_json_format(df_train,instruction_template)
test_data_litgpt = convert_to_json_format(df_test,instruction_template)

# Guardar a archivos JSON
with open('data/train_data/train_data.json', 'w', encoding='utf-8') as f:
    json.dump(train_data_litgpt, f, ensure_ascii=False, indent=4)

with open('data/train_data/test_data.json', 'w', encoding='utf-8') as f:
    json.dump(test_data_litgpt, f, ensure_ascii=False, indent=4)

print("Archivos guardados: data/train_data.json y data/test_data.json")

Archivos guardados: data/train_data.json y data/test_data.json
